In [ ]:
import torch 
import numpy as np
import config
from Plotting.myplt import plot_lines,plot_lines_with_colors
import tslearn
from tslearn import clustering
import matplotlib
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
import numba
from numba import jit
from sklearn.cluster import DBSCAN, OPTICS,KMeans,AgglomerativeClustering
from sklearn import mixture
import networkx as nx
import graph_layout
from sklearn import manifold
# matplotlib.use('TkAgg')

import myocyte_de_config
import config as config
import numpy as np
from myocyte_gen_des import F_vec
from scipy.integrate import odeint
from Plotting.myplt import *
from aml.time_mesuarment import timeit
from pprint import pprint as Print
from matplotlib import collections as matcoll
from numba import jit
from ParsingSystem.parse_and_build import TableGenerator
from sklearn import decomposition

@jit(nopython=True)
def code_distance(x1:np.array,x2:np.array):
    d = 0
    for i in range(len(x1)):
        d += int(x1[i]!=x2[i])
    return d/len(x1)

def get_distance_matrix(X, metric):
    N_ = len(X)
    d_m = np.zeros(shape=(N_,N_))
    for i in range(N_-1):
        print('\r{}/{}'.format(i,N_-2),end='')
        for j in range(i+1,N_):
            d_ = metric(X[i],X[j])
            d_m[i][j] = d_
            d_m[j][i] = d_
    print('')
    return d_m
def get_all_distances(X, metric):
    N_ = len(X)
    distances_ = np.zeros(shape=(int(N_*(N_-1)/2),))
    k_ = 0
    for i in range(N_-1):
        print('\r{}/{}'.format(i,N_-2),end='')
        for j in range(i+1,N_):
            distances_[k_] = metric(X[i],X[j])
            k_ += 1
    print('')
    return distances_
def plot_float_distribution(data,fig_size=(4,3),title=''):
    fig, ax = plt.subplots()
    fig.set_size_inches(fig_size[0],fig_size[1])
    x = []
    for i in range(len(data)):
        if np.isnan(data[i]):
            continue
        else:
            x.append(data[i])
    u_vs = np.unique(x)

    if len(x) == 0:
        ax.set_title(title +' is empty data')
    elif len(u_vs)==1:
        ax.set_title(title + ' all data is repeated with value: {}'.format(u_vs[0]))
    else:
        x = np.asarray(x)
        q25, q75 = np.percentile(x, [25, 75])
        bins = 0
        if q25==q75:
            bins = np.minimum(100,len(u_vs))
        else:
            bin_width = 2 * (q75 - q25) * len(x) ** (-1 / 3)
            bins = np.minimum(100, round((np.max(x) - np.min(x)) / bin_width))
        nan_rate = np.sum(np.isnan(data))/len(data)
        ax.set_title(title+'. n of unique values {}'.format(len(u_vs)))
        ax.set_xlabel('nan rate {}'.format(nan_rate))
        density,bins = np.histogram(x,bins=bins,density=True)
        unity_density = density / density.sum()
        widths = bins[:-1] - bins[1:]
        ax.bar(bins[1:], unity_density,width=widths)

    return fig,ax

In [ ]:
X = torch.load(r'/home/user/gr_lab/data/labels_after_clustering.txt')
distances = get_all_distances(X,metric=code_distance)
distance_matrix = get_distance_matrix(X,metric=code_distance)

In [ ]:
plot_float_distribution(distances)
print(np.sort(distances)[:30])
print(np.min(distances))
print(np.percentile(distances,1))
print(np.percentile(distances,2))
print(np.percentile(distances,3))
print(np.percentile(distances,4))
print(np.percentile(distances,5))
print(np.percentile(distances,25))
print(np.percentile(distances,50))
print(np.percentile(distances,75))
print(np.percentile(distances,99))
plt.show()

In [ ]:
clustering_alg_ = OPTICS(min_samples=4,metric=code_distance)
clustering_alg_.fit(X)
clusters_labels_ = clustering_alg_.labels_
unique_labels_ = np.unique(clusters_labels_)
N_ = len(np.unique(clusters_labels_))

In [ ]:
print(N_)
cmap = matplotlib.colors.ListedColormap(sns.color_palette("bright", N_).as_hex())
colors_ = []
for i in range(len(clusters_labels_)):
    label_ = clusters_labels_[i]
    if label_ == -1: 
        colors_.append((1.0,1.0,1.0,0.0))
    else:
        colors_.append(cmap(label_))
unique_labels_colors_ = []
for i in range(len(unique_labels_)):
    color_ = 0
    if unique_labels_[i] == -1:
        color_ = (1.0,1.0,1.0,0.0)
    else:
        color_ = cmap(unique_labels_[i])
    unique_labels_colors_.append(color_)

In [ ]:
print((np.sum(clusters_labels_==-1)))

visualize clusters in plane

In [ ]:
gr_model = graph_layout.GraphOnAPlane(distance_matrix,size_of_output_space = 33)
gr_model.fit()
gr_model.plot_loss()
positions = gr_model.get_pos()

In [ ]:
dim_reduction_alg1 = manifold.TSNE(n_components=2)
dim_reduction_alg2 = decomposition.PCA(n_components=2)
positions_1 = dim_reduction_alg1.fit_transform(positions)
positions_2 = dim_reduction_alg2.fit_transform(positions)
N_ = len(positions)
fig,ax1 = graph_layout.cluster_plot((5,5), positions_1, colors_of_points=colors_,color_for_labels=unique_labels_colors_,labels_for_colors=unique_labels_)
fig,ax2 = graph_layout.cluster_plot((5,5), positions_2, colors_of_points=colors_,color_for_labels=unique_labels_colors_,labels_for_colors=unique_labels_)
ax1.set_title('t-SNE')
ax2.set_title('PCA')
plt.show()

In [ ]:
fig.savefig('./tmp_plot.png',dpi= 200)

plot elements from clusters

In [ ]:
dataset = torch.load(config.dataset_for_time_clustering_path)
Theta = dataset['params']
Y =  dataset['solutions']
names = dataset['names']
index_by_name = dataset['index_by_name']
name_by_index = dataset['name_by_index']
start_point = dataset['start_point']
t_0 = dataset['t_0']
t_end = dataset['t_end']
tau = dataset['tau']
to_new_y_name = torch.load(config.myocyte_map_from_old_y_name_to_new_name_dict_filename)
print('Theta {}\tY {}'.format(Theta.shape, Y.shape))
print(names)

In [ ]:
for i in tqdm(range(len(unique_labels_))):
    label = unique_labels_[i]
    # if label==-1:
    #     continue
    # get random example 
    examples_indexes = np.argwhere(clusters_labels_==label).flatten()
    random_pos = np.random.randint(low=0,high=len(examples_indexes))
    example_index = examples_indexes[random_pos]
    print(label)
    print(examples_indexes)
    print(example_index)
    example = Y[example_index,:,:]
    params_vec = Theta[example_index]
    # plot solutions 
    N_time = int((t_end-t_0)/tau)+1
    time_grid = np.linspace(start=t_0, stop=t_end, num=N_time)
    solutions = odeint(func=F_vec, y0=start_point, t=time_grid, args=(params_vec,), full_output=False)
    fig = init_figure()
    fig = plot_solutions(fig, solutions, time_grid, to_new_y_name)
    FcarbTimeSeries = np.asarray([myocyte_de_config.F_carb(time_grid[i]) for i in range(len(time_grid))])
    FfatTimeSeries = np.asarray([myocyte_de_config.F_fat(time_grid[i]) for i in range(len(time_grid))])
    FprotTimeSeries = np.asarray([myocyte_de_config.F_prot(time_grid[i]) for i in range(len(time_grid))])
    add_line_to_fig(fig, time_grid,FprotTimeSeries,'F_{prot}')
    add_line_to_fig(fig, time_grid,FfatTimeSeries,'F_{fat}')
    add_line_to_fig(fig, time_grid,FcarbTimeSeries,'F_{carb}')
    fig.show()
    save_fig_to_html(fig,path=config.myo_path_to_html,filename='solution_{}.html'.format(i))
# label = unique_labels_[57]
# examples_indexes = np.argwhere(clusters_labels_==label).flatten()
# print(examples_indexes)
# for ex_index in examples_indexes:
#     example_index = ex_index
#     example = Y[example_index,:,:]
#     params_vec = Theta[example_index]
#     # plot solutions 
#     N_time = int((t_end-t_0)/tau)+1
#     time_grid = np.linspace(start=t_0, stop=t_end, num=N_time)
#     solutions = odeint(func=F_vec, y0=start_point, t=time_grid, args=(params_vec,), full_output=False)
#     fig = init_figure()
#     fig = plot_solutions(fig, solutions, time_grid, to_new_y_name)
#     FcarbTimeSeries = np.asarray([myocyte_de_config.F_carb(time_grid[i]) for i in range(len(time_grid))])
#     FfatTimeSeries = np.asarray([myocyte_de_config.F_fat(time_grid[i]) for i in range(len(time_grid))])
#     FprotTimeSeries = np.asarray([myocyte_de_config.F_prot(time_grid[i]) for i in range(len(time_grid))])
#     add_line_to_fig(fig, time_grid,FprotTimeSeries,'F_{prot}')
#     add_line_to_fig(fig, time_grid,FfatTimeSeries,'F_{fat}')
#     add_line_to_fig(fig, time_grid,FcarbTimeSeries,'F_{carb}')
#     fig.show()
#     save_fig_to_html(fig,path=config.myo_path_to_html,filename='solution_{}.html'.format(i))